In [26]:
import pandas as pd

In [27]:
data_filename0 = "/content/train0.csv"

In [28]:
data_filename1 = "/content/train1.csv"

In [29]:
df0 = pd.read_csv(data_filename0)

In [30]:
df1 = pd.read_csv(data_filename1)

In [31]:
df0.head(5)

,timestamp,image_idx,latitude,longitude,altitude
0,09-00-31-289685,0,17.496252,78.415602,485.792053
1,09-00-31-351936,1,17.496252,78.415602,485.788849
2,09-00-31-417976,2,17.496252,78.415602,485.783875
3,09-00-31-485167,3,17.496252,78.415602,485.786102
4,09-00-31-556803,4,17.496252,78.415601,485.782440


In [32]:
df1.head(100)

,timestamp,image_idx,latitude,longitude,altitude
0,13-00-06-217517,0,17.496380,78.415667,486.050812
1,13-00-06-284378,1,17.496378,78.415667,486.059143
2,13-00-06-351798,2,17.496377,78.415666,486.063812
3,13-00-06-417797,3,17.496375,78.415666,486.058350
4,13-00-06-485831,4,17.496373,78.415665,486.048553
...,...,...,...,...,...
95,13-00-12-551922,95,17.496160,78.415539,486.527222
96,13-00-12-620200,96,17.496157,78.415537,486.533264
97,13-00-12-685178,97,17.496153,78.415534,486.536652
98,13-00-12-754772,98,17.496150,78.415532,486.534332


In [33]:
df0.shape

(2211, 5)

In [34]:
import pandas as pd



def convert_timestamp_to_seconds(timestamp):
    try:
        # Split the timestamp into components
        components = timestamp.split('-')
        if len(components) != 4:
            raise ValueError(f"Timestamp format is incorrect: {timestamp}")

        hours, minutes, seconds, microseconds = components

        # Convert components to integers
        hours = int(hours)
        minutes = int(minutes)
        seconds = int(seconds)
        microseconds = int(microseconds)

        # Convert hours, minutes, and microseconds to seconds
        total_seconds = (hours * 3600) + (minutes * 60) + seconds + (microseconds / 1e6)

        return total_seconds
    except Exception as e:
        print(f"Error processing timestamp {timestamp}: {e}")
        return None

# Apply the conversion function to the timestamp column
df1['timestamp_in_seconds'] = df1['timestamp'].apply(convert_timestamp_to_seconds)


In [35]:
import pandas as pd



def convert_timestamp_to_seconds(timestamp):
    try:
        # Split the timestamp into components
        components = timestamp.split('-')
        if len(components) != 4:
            raise ValueError(f"Timestamp format is incorrect: {timestamp}")

        hours, minutes, seconds, microseconds = components

        # Convert components to integers
        hours = int(hours)
        minutes = int(minutes)
        seconds = int(seconds)
        microseconds = int(microseconds)

        # Convert hours, minutes, and microseconds to seconds
        total_seconds = (hours * 3600) + (minutes * 60) + seconds + (microseconds / 1e6)

        return total_seconds
    except Exception as e:
        print(f"Error processing timestamp {timestamp}: {e}")
        return None

# Apply the conversion function to the timestamp column
df0['timestamp_in_seconds'] = df0['timestamp'].apply(convert_timestamp_to_seconds)


In [36]:
df0.head(5)

,timestamp,image_idx,latitude,longitude,altitude,timestamp_in_seconds
0,09-00-31-289685,0,17.496252,78.415602,485.792053,32431.289685
1,09-00-31-351936,1,17.496252,78.415602,485.788849,32431.351936
2,09-00-31-417976,2,17.496252,78.415602,485.783875,32431.417976
3,09-00-31-485167,3,17.496252,78.415602,485.786102,32431.485167
4,09-00-31-556803,4,17.496252,78.415601,485.782440,32431.556803


In [37]:
df0.dtypes

timestamp                object
image_idx                 int64
latitude                float64
longitude               float64
altitude                float64
timestamp_in_seconds    float64
dtype: object

In [38]:
df1.shape

(2166, 6)

In [39]:
df0.shape

(2211, 6)

In [40]:
df0= df0.iloc[:-45]

In [41]:
df0.shape

(2166, 6)

In [42]:
import pandas as pd
import numpy as np

# Function to calculate distance between two points on Earth (Haversine formula)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3  # Radius of the Earth in meters
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

# Function to calculate 3D distance including altitude
def distance_3d(lat1, lon1, alt1, lat2, lon2, alt2):
    distance_2d = haversine(lat1, lon1, lat2, lon2)
    distance_3d = np.sqrt(distance_2d**2 + (alt2 - alt1)**2)
    return distance_3d

# Function to calculate Time to Collision (TTC)
def calculate_ttc(distance, speed):
    if speed == 0:
        return np.inf
    return distance / speed

# Set a threshold distance in meters for collision warning
threshold_distance = 5  # Example threshold

# Initialize lists to store results
distances = []
time_diffs = []
relative_speeds = []
ttcs = []
warnings = []

# Iterate over each row in the dataframes
for index, row in df0.iterrows():
    lat1, lon1, alt1, timestamp1 = row[['latitude', 'longitude', 'altitude', 'timestamp_in_seconds']]
    lat2, lon2, alt2, timestamp2 = df1.iloc[index][['latitude', 'longitude', 'altitude', 'timestamp_in_seconds']]

    # Calculate distance and time difference
    distance = distance_3d(lat1, lon1, alt1, lat2, lon2, alt2)
    time_diff = timestamp2 - timestamp1

    # Calculate relative speed and TTC
    relative_speed = distance / time_diff
    ttc = calculate_ttc(distance, relative_speed)

    # Check for warnings
    warning = distance < threshold_distance

    # Store results
    distances.append(distance)
    time_diffs.append(time_diff)
    relative_speeds.append(relative_speed)
    ttcs.append(ttc)
    warnings.append(warning)

# Add results to the dataframe
df0['distance'] = distances
df0['time_diff'] = time_diffs
df0['relative_speed'] = relative_speeds
df0['ttc'] = ttcs
df0['warning'] = warnings

# Output the results
print(df0)


            timestamp  image_idx   latitude  longitude    altitude  \
0     09-00-31-289685          0  17.496252  78.415602  485.792053   
1     09-00-31-351936          1  17.496252  78.415602  485.788849   
2     09-00-31-417976          2  17.496252  78.415602  485.783875   
3     09-00-31-485167          3  17.496252  78.415602  485.786102   
4     09-00-31-556803          4  17.496252  78.415601  485.782440   
...               ...        ...        ...        ...         ...   
2161  09-02-55-342241       2161  17.493366  78.410767  494.634247   
2162  09-02-55-414774       2162  17.493369  78.410764  494.658600   
2163  09-02-55-479782       2163  17.493372  78.410762  494.656708   
2164  09-02-55-545942       2164  17.493377  78.410757  494.683197   
2165  09-02-55-608403       2165  17.493380  78.410755  494.714874   

      timestamp_in_seconds   distance     time_diff  relative_speed  \
0             32431.289685  15.878296  14374.927832        0.001105   
1             324

In [49]:
df0.head(5)

,timestamp,image_idx,latitude,longitude,altitude,timestamp_in_seconds,distance,time_diff,relative_speed,ttc,warning
0,09-00-31-289685,0,17.496252,78.415602,485.792053,32431.289685,15.878296,14374.927832,0.001105,14374.927832,False
1,09-00-31-351936,1,17.496252,78.415602,485.788849,32431.351936,15.612248,14374.932442,0.001086,14374.932442,False
2,09-00-31-417976,2,17.496252,78.415602,485.783875,32431.417976,15.474800,14374.933822,0.001077,14374.933822,False
3,09-00-31-485167,3,17.496252,78.415602,485.786102,32431.485167,15.339833,14374.932630,0.001067,14374.932630,False
4,09-00-31-556803,4,17.496252,78.415601,485.782440,32431.556803,15.056992,14374.929028,0.001047,14374.929028,False


In [47]:
df0['warning'].value_counts()

warning
False    2135
True       31
Name: count, dtype: int64